 # Opis zadania
 W tym notatniku stworzymy system analityczno-raportowy, mający wspomagać przyszłe decyzje biznesowe. Dzięki przeprowadzonej wcześniej analizie eksploracyjnej danych, w trakcie spotkania prezentującego wykonaną pracę postanowiono w pierwszym kroku stworzyć następujące wizualizacje danych:
 - Wyświetlenie TOP 10 tras samolotów pod względem najniższego odsetka opóźnień w formie tabeli,
 - Wyświetlenie porównania roku 2019 oraz 2020 w formie wykresu słupkowego:
     - miesiąc do miesiąca,
     - dzień tygodnia do dnia tygodnia,
 - Wyświetlenie danych dzień po dniu w formie szeregu czasowego.

Raport powinien składać się z trzech stron - `TOP report`, `Comparision` i `Day by day reliability`.

Pracując na podstawie wcześniej zdefiniowanych widoków, nie musimy się tutaj martwić o przetwarzanie oraz procesowanie danych. Co więcej, wszystkie niuanse techniczne w postaci liczby wierszy, wykonywanych złączeń, filtracji są przeniesione na bazę danych. Z technicznego punktu widzenia, pomiędzy serwisem a bazą danych dochodzi do przesyłania mniejszej ilości danych.

Dopuszczamy jednak pewne aktualizacje co do ich struktury przykładowo poprzez _pivotowanie_, czyli obranie i doprowadzanie do postaci tej znanej z tabel przestawnych.

> **Uwaga:**  
> Przy pracy nad poprawkami w dashboardzie, pamiętaj, że aby odświeżyć stronę po wprowadzonych zmianach, należy **całość** uruchomić ponownie.

W tej części projektu końcowego wcielasz się ponownie w rolę BI Developera, który ma za zadanie stworzyć dashboard zgodny z wytycznymi biznesowymi dostarczonymi przez klienta. Pamiętaj, że osoba na tym stanowisku często ma kontakt z biznesem więc musi umieć przekazać informację o danych, które zawarła na dashboardzie by te zaangażowały odbiorców.


 Tutaj zaimportuj potrzebne biblioteki

In [ ]:
import psycopg2
from psycopg2 import connect
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import html
from dash import dcc
from dash import dash_table
from dash.dependencies import Input, Output

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
username = 'postgres'
password = 'postgres'

host = 'localhost'
database = 'airlines'
port = 5432

url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(url)

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [ ]:
con=psycopg2.connect(
    database=database,
    user=username,
    password=password,
    host=host,
    
)
cursor = con.cursor()
print("Połączenie udane.")

 # Stworzenie tabeli dla `TOP 10 routes`
 W tym miejscu przygotujemy tabelkę oraz komponent, który zostanie następnie umieszczony w raporcie. Tabela powinna się składać z następujących kolumn:
 - Origin,
 - Destination,
 - Year,
 - Reliability,
 - Rank.

Wartości procentowe zaś powinny być postaci np. 13.87%.

Z tego względu konieczna będzie zmiana nazw kolumn dla wyników raportowania.

Wskazówki:
 - `Python - analiza danych > Dzień 5 - Pandas > Obróbka danych - część 2 > apply`,
 - [How to format percentage in python](https://stackoverflow.com/questions/5306756/how-to-print-a-percentage-value-in-python).

 Tutaj zaczytaj dane do ramki danych `top_routes_df` z widoku `reporting.top_reliability_roads`
 > Jeśli pojawi się komunikat `UserWarning: pandas only support SQLAlchemy`, z naszego punktu widzenia, możemy go śmiało zignorować.

In [ ]:
query = "SELECT * FROM reporting.top_reliability_roads"
top_routes_df = pd.read_sql_query(query, engine)

In [ ]:
top_routes_df

 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [ ]:
top_routes_df['reliability'] = top_routes_df['reliability'].apply(lambda x: f"{(x * 100):.2f}%")

In [ ]:
top_routes_df

 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [ ]:
top_routes_df.rename(columns={
    'origin_airport_name': 'Origin',
    'dest_airport_name': 'Destination',
    'year': 'Year',
    'reliability': 'Reliability',
    'nb': 'Rank'
},inplace=True)

In [ ]:
top_routes_df

 # Porównanie rok do roku (YoY) 2019 vs. 2020
 W tym miejscu stworzymy wykres oraz komponent, który zostanie następnie umieszczony w raporcie. Wykres powinien przedstawiać porównanie lat 2019 oraz 2020 po miesiącach względem zmiennej `reliability`.

 > Na wykresie chcemy również przedstawić dane, które nie mają porównania tj. od kwietnia do grudnia 2019.

W tym celu wykonamy poniższe zadania:
 - Przekształcimy tabelkę do formy tabeli przestawnej
 - Stworzymy wykres słupkowy porównujący dane rok do roku
 - Opakujemy stworzony wykres w komponent `Dash`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 1 > Budowanie wykresów plotly`,
 - `Python - analiza danych > Dzień 6 - Pandas c.d > Pivot`.

 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [ ]:
query = "SELECT * FROM reporting.year_to_year_comparision"
yoy_comparision_df = pd.read_sql_query(query, engine)

In [ ]:
yoy_comparision_df

 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [ ]:
yoy_comparision_to_plot_df = pd.pivot_table(yoy_comparision_df, values='reliability', index='month', columns='year', aggfunc='mean')

In [ ]:
yoy_comparision_to_plot_df

 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [ ]:
yoy_comparision_fig = go.Figure()

yoy_comparision_fig.add_trace(go.Bar(
    x=yoy_comparision_to_plot_df.index,
    y=yoy_comparision_to_plot_df[2019] * 100,
    name='2019 reliability',
    marker_color='blue',
    text=yoy_comparision_to_plot_df[2019].apply(lambda x: f"{(x * 100):.2f}%"),
    textposition='auto'
))

yoy_comparision_fig.add_trace(go.Bar(
    x=yoy_comparision_to_plot_df.index,
    y=yoy_comparision_to_plot_df[2020] * 100,
    name='2020 reliability',
    marker_color='red',
    text=yoy_comparision_to_plot_df[2020].apply(lambda x: f"{(x * 100):.2f}%"),
    textposition='auto'
))

yoy_comparision_fig.update_layout(
    title='Comparison of Reliability each month for 2019 and 2020',
    xaxis_title='Month',
    yaxis_title='Reliability (%)',
    barmode='group',
    xaxis=dict(
        tickvals=list(range(1, 13))
    )
)

 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020
 Wykonaj analogiczne do poprzedniego kroku, używając jednak w tym momencie danych z widoku `reporting.day_to_day_comparision`

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [ ]:
query = "SELECT * FROM reporting.day_to_day_comparision"
day_to_day_comparision_df = pd.read_sql_query(query, engine)

In [ ]:
day_to_day_comparision_df

 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [ ]:
day_to_day_comparision_to_plot_df = pd.pivot_table(day_to_day_comparision_df, values='reliability', index='day_of_week', columns='year', aggfunc='mean')

In [ ]:
day_to_day_comparision_to_plot_df

 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [ ]:
day_to_day_comparision_fig = go.Figure()

day_to_day_comparision_fig.add_trace(go.Bar(
    x=day_to_day_comparision_to_plot_df.index,
    y=day_to_day_comparision_to_plot_df[2019] * 100,
    name='2019 reliability',
    marker_color='blue',
    text=day_to_day_comparision_to_plot_df[2019].apply(lambda x: f"{(x * 100):.2f}%"),
    textposition='auto'
))

day_to_day_comparision_fig.add_trace(go.Bar(
    x=day_to_day_comparision_to_plot_df.index,
    y=day_to_day_comparision_to_plot_df[2020] * 100,
    name='2020 reliability',
    marker_color='red',
    text=day_to_day_comparision_to_plot_df[2020].apply(lambda x: f"{(x * 100):.2f}%"),
    textposition='auto'
))

day_to_day_comparision_fig.update_layout(
    title='Comparison of Reliability each day of week for 2019 and 2020',
    xaxis_title='Day of week',
    yaxis_title='Reliability (%)',
    barmode='group'
)

 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [ ]:
query = "SELECT * FROM reporting.day_by_day_reliability"
day_by_day_reliability_df = pd.read_sql_query(query, engine)

In [ ]:
day_by_day_reliability_df.info()

 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [ ]:
day_by_day_reliability_df['date'] = pd.to_datetime(day_by_day_reliability_df['date'], errors='coerce')
day_by_day_reliability_df = day_by_day_reliability_df.dropna(subset=['date'])

# Dodanie kolumny 'Year' oraz 'Month-Day' (miesiąc-dzień)
day_by_day_reliability_df['Year'] = day_by_day_reliability_df['date'].dt.year
day_by_day_reliability_df['Month-Day'] = day_by_day_reliability_df['date'].dt.strftime('%m-%d')

# Podział danych na rok 2019 i 2020
data_2019 = day_by_day_reliability_df[day_by_day_reliability_df['Year'] == 2019].copy()
data_2020 = day_by_day_reliability_df[day_by_day_reliability_df['Year'] == 2020].copy()



In [ ]:
# Tworzenie wykresu szeregu czasowego
day_by_day_reliability_fig = go.Figure()

# Dodawanie danych dla 2019
day_by_day_reliability_fig.add_trace(go.Scatter(
    x=data_2019['Month-Day'],
    y=data_2019['reliability'] * 100,  # Przekształcenie na procenty
    mode='lines',
    name='2019 reliability',
    line=dict(color='blue'),
    hovertemplate='<b>%{x}</b><br>Reliability: %{y:.2f} %<extra></extra>'
))

# Dodawanie danych dla 2020
day_by_day_reliability_fig.add_trace(go.Scatter(
    x=data_2020['Month-Day'],
    y=data_2020['reliability'] * 100,  # Przekształcenie na procenty
    mode='lines',
    name='2020 reliability',
    line=dict(color='red'),
    hovertemplate='<b>%{x}</b><br>Reliability: %{y:.2f} %<extra></extra>'
))

# Konfiguracja wykresu
day_by_day_reliability_fig.update_layout(
    title='Daily Reliability Comparison for 2019 and 2020',
    xaxis_title='Day of the Year',
    yaxis_title='Reliability (%)',
    legend_title='Legend',
    xaxis_rangeslider_visible=False,
    xaxis=dict(
        showticklabels=False,
        tickmode='array',
        tickvals=data_2019['Month-Day'],  # Wyświetlanie wszystkich dni na osi x
        ticktext=data_2019['Month-Day']
    )
)

day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

 ## Stworzenie layout dla widoku `TOP 10 routes`
 W tym momencie przekonwertujemy zmienną `top_routes_df` w postać `Dash.DataTable`. Dodatkowo, dla czytelności, dodamy nagłówek opisujący co tabela przedstawia.

 Nagłówek wystylizuj używając poniższych wymagań:
 - tag: `H3`,
 - czcionka: `verdana`,
 - kolor: '#4444`,
 - tekst do wyświetlenia: `TOP 10 reliability routes in 2019 and 2020`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 3 > Dash - datatable`,
 - Dokmentacja metody `Pandas` - [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html).

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [ ]:
top_routes_table = dash_table.DataTable(
    id='top-routes-table',
    columns=[{"name": i, "id": i} for i in top_routes_df.columns],
    data=top_routes_df.to_dict('records'),
    style_table={
        'overflowX': 'auto'
    },
    style_header={
        'backgroundColor': '#f9f9f9',
        'fontWeight': 'bold'
    },
    style_cell={
        'textAlign': 'left',
        'padding': '5px'
    }
)

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [ ]:
top_routes_page_title = html.H3(
    "TOP 10 reliability routes in 2019 and 2020",
    style={
        'fontFamily': 'Verdana',
        'color': '#4444',
        'textAlign': 'center',
        'marginBottom': '20px'
    }
)

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [ ]:
top_routes_layout = html.Div([
    # Nagłówek
    top_routes_page_title,
    
    # Tabela danych
    top_routes_table
], style={
    'padding': '20px',  # Padding around the content
    'backgroundColor': '#f8f9fa'  # Background color of the layout
})

 ## Stworzenie layout dla widoku `Comparision`
 W tym momencie opakujemy zmienne `yoy_comparision_fig` oraz `day_to_day_comparision_fig` do komponentów `Dash`, a następnie stworzymy odpowiedni `layout` dla strony `comparision`. Wyniki zapiszemy do zmiennej `comparision_layout`.

 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [ ]:
yoy_comparision_component = dcc.Graph(
    id='yoy-comparison-graph',  # Unikalny identyfikator dla komponentu
    figure=yoy_comparision_fig  # Wykres, który ma być wyświetlany
)

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [ ]:
# Opakowanie wykresu day_to_day_comparision_fig w komponent Dash
day_to_day_comparision_component = dcc.Graph(
    id='day-to-day-comparison-graph',  # Unikalny identyfikator dla komponentu
    figure=day_to_day_comparision_fig  # Wykres, który ma być wyświetlany
)

 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [ ]:
comparision_layout = html.Div([
    # Dodajemy oba wykresy do layoutu
    html.Div([
        html.H3('Year-over-Year Reliability Comparison', style={'font-family': 'Verdana', 'color': '#4444'}),
        yoy_comparision_component
    ], style={'margin-bottom': '30px'}),
    
    html.Div([
        html.H3('Day-by-Day Reliability Comparison', style={'font-family': 'Verdana', 'color': '#4444'}),
        day_to_day_comparision_component
    ])
], style={
    'padding': '20px',  # Padding around the content
    'backgroundColor': '#f8f9fa'  # Background color of the layout
})

 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [ ]:
# Opakowanie wykresu day_by_day_reliability_fig w komponent Dash
day_by_day_reliability_component = dcc.Graph(
    id='day-by-day-reliability-graph',  # Unikalny identyfikator dla komponentu
    figure=day_by_day_reliability_fig  # Wykres, który ma być wyświetlany
)

# Stworzenie layoutu dla widoku day_by_day
day_by_day_layout = html.Div([
    html.H3('Day-by-Day Reliability for 2019 and 2020', style={'font-family': 'Verdana', 'color': '#4444'}),
    day_by_day_reliability_component
], style={
    'padding': '20px',  # Padding around the content
    'backgroundColor': '#f8f9fa'  # Background color of the layout
})

 # Konfigurowanie aplikacji
 Posiadamy już wszystkie niezbędne komponenty do stworzenia oraz uruchomienia aplikacji. Naszym zadaniem w tym miejscu będzie odpowiednie jej skonfigurowanie, aby obsługiwała więcej niż jedną stronę.

 Chcemy, aby możliwe było poruszanie się pomiędzy poniższymi stronami:
 - `TOP report` - strona domyślna,
 - `Comparision` - strona dostępna pod adresem: `comparision_reporting`,
 - `Day by day reporting` - strona dostępna pod adresem: `day_by_day_reporting`

Wskazówki:
 - `Wizualizacja danych > Dzień 4 > Callback context`,
 - `Wizualizacja danych > Dzień 4 > Aplikacja multipage`.
 - Używając komponentu [html.Button](https://community.plotly.com/t/button-with-link/11809), możesz łatwo stworzyć ładniejsze linki do poszczególnych stron.

 Tutaj przygotuj layout aplikacji

In [ ]:
app = dash.Dash(__name__, suppress_callback_exceptions=True)

In [ ]:
app.layout = html.Div([
    # Nawigacja
    html.Div([
        html.Button('TOP Report', id='top-report-button', n_clicks=0),
        html.Button('Comparision', id='comparision-button', n_clicks=0),
        html.Button('Day by Day Reporting', id='day-by-day-button', n_clicks=0)
    ], style={'margin-bottom': '20px', 'padding': '10px'}),
    
    # Dynamiczny kontener na treści stron
    html.Div(id='page-content')
])

In [ ]:
yoy_comparision_component = dcc.Graph(figure=yoy_comparision_fig)
day_to_day_comparision_component = dcc.Graph(figure=day_to_day_comparision_fig)

In [ ]:
top_routes_layout = html.Div([
    top_routes_page_title,
    top_routes_table
])


In [ ]:
comparision_layout = html.Div([
    html.Div([
        yoy_comparision_component
    ], style={'width': '100%', 'margin-bottom': '20px'}),  # Stylizacja dla pierwszego wykresu
    html.Div([
        day_to_day_comparision_component
    ], style={'width': '100%'})  # Stylizacja dla drugiego wykresu
])

In [ ]:
# Layout dla Day by Day Report
day_by_day_layout = html.Div([
    html.H3('Day by Day Reporting'),
    dcc.Graph(figure=day_by_day_reliability_fig)  # Przykładowy wykres
])

 Tutaj stwórz procedurę odpowiadającą za nawigację pomiędzy poszczególnymi stronami

In [ ]:
@app.callback(
    Output('page-content', 'children'),
    [Input('top-report-button', 'n_clicks'),
     Input('comparision-button', 'n_clicks'),
     Input('day-by-day-button', 'n_clicks')]
)
def display_page(top_report_clicks, comparision_clicks, day_by_day_clicks):
    
    if top_report_clicks is None:
        top_report_clicks = 0
    if comparision_clicks is None:
        comparision_clicks = 0
    if day_by_day_clicks is None:
        day_by_day_clicks = 0

    # Sprawdzanie, który przycisk ma najwyższą liczbę kliknięć
    if top_report_clicks >= comparision_clicks and top_report_clicks >= day_by_day_clicks:
        return top_routes_layout
    elif comparision_clicks >= top_report_clicks and comparision_clicks >= day_by_day_clicks:
        return comparision_layout
    elif day_by_day_clicks >= top_report_clicks and day_by_day_clicks >= comparision_clicks:
        return day_by_day_layout
    
    return top_routes_layout

 Tutaj uruchom aplikację

In [ ]:
# Uruchomienie aplikacji
if __name__ == '__main__':
    app.run_server(debug=True)

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.